In [1]:
import pandas as pd
bureau_data=pd.read_csv("IS453 Group Assignment - Bureau Data.csv")
application_data=pd.read_csv("IS453 Group Assignment - Application Data.csv")


In [ ]:

#choosing columns to keep 
application_data_cleaned = application_data[["SK_ID_CURR",'TARGET','FLAG_OWN_CAR','FLAG_OWN_REALTY',
                                             'CNT_CHILDREN','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE',
                                             'NAME_INCOME_TYPE','DAYS_EMPLOYED','OWN_CAR_AGE','FLAG_MOBIL','FLAG_CONT_MOBILE',
                                             'CNT_FAM_MEMBERS','REG_REGION_NOT_LIVE_REGION','EXT_SOURCE_1','AMT_REQ_CREDIT_BUREAU_MON']]
bureau_data_cleaned=bureau_data[["SK_ID_CURR","SK_ID_BUREAU","CREDIT_ACTIVE",'CREDIT_DAY_OVERDUE','AMT_CREDIT_MAX_OVERDUE',
                                 'CNT_CREDIT_PROLONG','AMT_CREDIT_SUM','AMT_CREDIT_SUM_DEBT','AMT_CREDIT_SUM_OVERDUE',
                                 'CREDIT_TYPE','DAYS_CREDIT_UPDATE','AMT_ANNUITY']]

#choosing rows to keep based on automobile loan
app_filtered = application_data_cleaned[application_data_cleaned['FLAG_OWN_CAR'] == 'N']
bureau_filtered = bureau_data_cleaned[bureau_data_cleaned['CREDIT_TYPE'] != 'Car loan']

#flattening bureau data

# Aggregating Bureau Data for each application (SK_ID_CURR)
bureau_aggregated = bureau_filtered.groupby('SK_ID_CURR').agg({
    'SK_ID_BUREAU': 'count',  # Number of past loans
    'CREDIT_ACTIVE': lambda x: x.value_counts().idxmax(),  # Most frequent credit status
    'CREDIT_DAY_OVERDUE': 'mean',  # Average overdue days
    'AMT_CREDIT_MAX_OVERDUE': 'max',  # Max overdue amount
    'CNT_CREDIT_PROLONG': 'sum',  # Total prolongations
    'AMT_CREDIT_SUM': 'sum',  # Total outstanding credit
    'AMT_CREDIT_SUM_DEBT': 'mean',  # Average current debt
    'AMT_CREDIT_SUM_OVERDUE': 'max',  # Max overdue amount
    'CREDIT_TYPE': lambda x: x.value_counts().idxmax(),  # Most frequent credit type
    'DAYS_CREDIT_UPDATE': 'min',  # Most recent credit update
    'AMT_ANNUITY': 'mean',  # Average annuity payment
}).reset_index()

#feature engineering
# Rename SK_ID_BUREAU count column
bureau_aggregated.rename(columns={"SK_ID_BUREAU": "CREDIT_COUNT"}, inplace=True)

# Create a new feature for overdue credit count
bureau_aggregated['OVERDUE_CREDIT_COUNT'] = bureau_data[bureau_data['CREDIT_DAY_OVERDUE'] > 0].groupby('SK_ID_CURR')['SK_ID_BUREAU'].transform('count')

# Merge the aggregated bureau data with application data
merged_data = application_data_cleaned.merge(bureau_aggregated, on='SK_ID_CURR', how='left')

#number of rows
rows=merged_data.shape[0]
print(f"Number of rows after selection: {rows}")

#calculate percentage of bad loans after filtering
total_loans=len(app_filtered)
bad_loans=app_filtered[app_filtered['TARGET']==1].shape[0]
bad_loans_percentage=(bad_loans/total_loans)*100
print(f"Percentage of Bad Loans after filtering:{bad_loans_percentage:.2f}%")

#calculate percentage of bad loans in orig dataset
total_loans=len(application_data_cleaned)
bad_loans=application_data_cleaned[application_data_cleaned['TARGET']==1].shape[0]
bad_loans_percentage=(bad_loans/total_loans)*100
print(f"Percentage of Bad Loans in orig dataset:{bad_loans_percentage:.2f}%")


In [ ]:
merged_data